## Relaxation on Perovskite Structures using Perovs-IAP
This notebook would aid you in loading our fine-tuned M3GNET model Perovs-IAP, and relax a perovskite structure from the Materials Project. We also plot the relaxation trajectory which shows how the relaxation proceeds. Only use the installation commands if you are running on colab. Or else, make an evironment directly for better usage.


In [ ]:
#ignore warnings command
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Uninstall conflicting libraries
!pip uninstall -y torch torchaudio torchvision torchdata torchtune thinc numpy

# Install matgl and its dependencies
!pip install matgl

In [ ]:
!pip uninstall dgl -y
!pip install  dgl -f https://data.dgl.ai/wheels/torch-2.4/repo.html

In [ ]:
!pip install mp_api pymatgen ase

In [ ]:
from mp_api.client import MPRester
import ast
from matgl import load_model
from matgl.ext.ase import Relaxer
from pymatgen.core import Structure

First, let's get a perovskite structure from the Materials Project, you would need your API-key for this. Make sure to get a perovskite structure to obtain accurate results.

In [ ]:
API_KEY = "9J76AeTI0TGu0SUAJS0BafOasktJ2CzB" #get your API key from https://next-gen.materialsproject.org/api
with MPRester(API_KEY) as mpr:
    # Get a perovskite structure from Materials Project
    structure = mpr.get_structure_by_material_id("mp-540839", conventional_unit_cell=True)  #Orthorhombic CsPbI3, you can use whatever you like
    structure.to("POSCAR", fmt="poscar")

Let us load the structure, and the model, and start the relaxation.

Similiar to a standard DFT, you can adjust whether or not to relax the cell, or change the force tolerance.

In [ ]:
!git clone https://github.com/maitreyo18/Fine-tuned-Perovs-IAP-for-Unified-Perovskite-Structure-Optimization.git

In [ ]:
struct = Structure.from_file("POSCAR")
pot = load_model("Fine-tuned-Perovs-IAP-for-Unified-Perovskite-Structure-Optimization/Perovs-IAP_finetuned_bulk+defect+surface_2025_PES")
relaxer = Relaxer(pot, relax_cell=False)  # Set relax_cell to True if you want to relax the cell parameters

In [ ]:
relaxed_struct = relaxer.relax(struct, fmax=0.001, steps=100)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Set font style to Arial Narrow using the uploaded file
font_path = '/content/Fine-tuned-Perovs-IAP-for-Unified-Perovskite-Structure-Optimization/arialnarrow.ttf'
try:
    # Add the font to Matplotlib's font manager
    fm.fontManager.addfont(font_path)
    plt.rcParams['font.family'] = 'Arial Narrow'
except Exception as e:
    plt.rcParams['font.family'] = 'sans-serif'


# Set large font sizes
plt.rcParams['axes.titlesize'] = 24
plt.rcParams['axes.labelsize'] = 24
plt.rcParams['xtick.labelsize'] = 24
plt.rcParams['ytick.labelsize'] = 24
plt.rcParams['legend.fontsize'] = 24
plt.rcParams['figure.titlesize'] = 24

# Plot the trajectory side by side
fig, axes = plt.subplots(1, 2, figsize=(14, 8)) # Create a figure with 1 row and 2 columns

steps = range(len(relaxed_struct['trajectory'].energies))
energies = relaxed_struct['trajectory'].energies
forces = relaxed_struct['trajectory'].forces
force_magnitudes = [sum(f**2 for f in step_forces.flatten())**0.5 for step_forces in forces]

# Print final energy and force magnitude
print(f"Final Energy: {energies[-1]:.4f} eV")
print(f"Final Force Magnitude: {force_magnitudes[-1]:.4f} eV/Angstrom")

# Calculate and print energy difference per atom
initial_energy = energies[0]
final_energy = energies[-1]
num_atoms = len(relaxed_struct['final_structure'])
energy_difference_per_atom = (final_energy - initial_energy) / num_atoms
print(f"Energy Difference per Atom: {energy_difference_per_atom:.4f} eV/atom")


# Plot Energy Trajectory in the left subplot
axes[0].plot(steps, energies, marker='o', linestyle='-', color='blue')
axes[0].set_xlabel('Relaxation Step')
axes[0].set_ylabel('Energy (eV)')
axes[0].set_title('Energy Trajectory')
axes[0].grid(True)

# Plot Force Magnitude Trajectory in the right subplot
axes[1].plot(steps, force_magnitudes, marker='x', linestyle='--', color='red')
axes[1].set_xlabel('Relaxation Step')
axes[1].set_ylabel('Force Magnitude (eV/Angstrom)')
axes[1].set_title('Force Magnitude Trajectory')
axes[1].grid(True)

plt.tight_layout() # Adjust layout to prevent overlapping titles/labels
plt.show()

In [ ]:
relaxed_struct['final_structure'].to("POSCAR_relaxed", fmt="poscar")